# Description

In [1]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c54f2e7f40e0418cb540ed70e11f9722 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='SyrI4oCfA7rN6fy-o2SGHEDA-wxByeikJ5KBz9d6NP0Q',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_c54f2e7f40e0418cb540ed70e11f9722.get_object(Bucket='urgency-donotdelete-pr-fbcrmlm7w8reii',Key='911_calls_for_service.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Unnamed: 0,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,0,2015-07-13 10:41:00,Medium,CD,SEE TEXT,P151941002,0 N CALVERT ST,"(39.2899299,-76.6123462)"
1,1,2015-07-13 10:47:00,Medium,CD,911/NO VOICE,P151941003,600 E FAYETTE ST,"(39.2906737,-76.6071600)"
2,2,2015-07-13 10:42:00,Medium,CD,911/NO VOICE,P151941004,200 E BALTIMORE ST,"(39.2898910,-76.6120720)"
3,3,2015-07-13 10:45:00,Low,CD,PRKG COMPLAINT,P151941005,800 PARK AV,"(39.2985163,-76.6184754)"
4,4,2015-07-13 10:46:00,Medium,SW,AUTO THEFT,P151941006,3500 CLIFTON AV,"(39.3112130,-76.6763150)"


## Importing all the libraries

In [2]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Mapping priorities to numeric  values and converting the description of the emergency to string

In [3]:
df_data_1['priority'] = df_data_1['priority'].map({
    'Medium':2,
    'Low':1,
    'High':3
    })
df_data_1['priority'] = df_data_1['priority'].fillna(value = 0)

df_data_1['description'] = df_data_1['description'].astype('str')

## Creating a bag of words

In [4]:
def process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    cleaned_text = [text for text in nopunc.split() if text.lower() not in stopwords.words('english')]
    return cleaned_text
text_bow = CountVectorizer(analyzer = process).fit_transform(df_data_1['description'])
print("Success")


Success


## Splitting into training and test set

In [5]:
X_train,  X_test, Y_train,Y_test = train_test_split(text_bow, df_data_1['priority'] , test_size=0.25, random_state = 0)


## Using the multinomialnb classifier to predict the test values

In [7]:
classifier = MultinomialNB().fit(X_train, Y_train)
pred = classifier.predict(X_test)
cm = confusion_matrix(Y_test, pred)
print(cm)

[[ 69465   4650   7945    292]
 [  1660 134993  16823   5348]
 [  1090   9296 340365     77]
 [    70    415    667 106823]]


In [9]:
print(accuracy_score(Y_test, pred))
print(classification_report(Y_test, pred))

0.9309507856664271
              precision    recall  f1-score   support

         0.0       0.96      0.84      0.90     82352
         1.0       0.90      0.85      0.88    158824
         2.0       0.93      0.97      0.95    350828
         3.0       0.95      0.99      0.97    107975

   micro avg       0.93      0.93      0.93    699979
   macro avg       0.94      0.91      0.92    699979
weighted avg       0.93      0.93      0.93    699979

